# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [20]:
# imports
from openai import OpenAI
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown, update_display


In [2]:
# constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)

# Roche Galileo specific initialization
api_key=os.getenv("OPENAI_API_KEY")
base_url=os.getenv("GALILEO_AI_GATEWAY")

openai = OpenAI(
    api_key=api_key,
    base_url=base_url
)


In [14]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [19]:
system_prompt = "You are a programmer expert, your job is " \
"to assist with coding tasks, answer programming questions, " \
"and provide code examples. Respond in markdown."
system_prompt

'You are a programmer expert, your job is to assist with coding tasks, answer programming questions, and provide code examples. Respond in markdown.'

In [15]:
user_prompt = question

In [53]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# Get gpt-4o-mini to answer, without streaming

response = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=messages,
    # response_format={"type": "json_object"},
    # timeout=60  # seconds, adjust as needed
)

result = response.choices[0].message.content


In [55]:
# print (result)
display (Markdown(result))


The code you've provided is a Python expression that uses a generator to yield values from a set. Let's break it down step-by-step to understand what it does:

### Code Explanation

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **Set Comprehension**: 
    - The expression `{book.get("author") for book in books if book.get("author")}` is a **set comprehension**. Set comprehensions are similar to list comprehensions but create a set, which automatically eliminates duplicate entries.
    - `book.get("author")`: This calls the `get` method on each `book` dictionary to retrieve the value associated with the key `"author"`. If the key does not exist, it returns `None` (or some default value if specified).
    - `for book in books`: This iterates over each item in the `books` iterable, which is expected to be a list (or another iterable) of dictionaries, where each dictionary represents a book.
    - `if book.get("author")`: This is a condition that filters the books. It ensures that only books with a defined (non-falsy) author are included in the set.

2. **Yielding Values**:
    - The `yield from` statement is used in a generator function to yield all values from an iterable. In this case, it will yield all unique author names (since the comprehension returns a set) that are found in the `books` collection.
    - The `yield from` syntax makes it easier to yield from an iterable without having to use an explicit loop.

### Summary

- The code collects the unique authors from a list of book dictionaries.
- It filters out any books that do not have an author (i.e., those where `book.get("author")` returns `None` or some falsy value).
- It yields each unique author name from the generated set.

### Use Case

This kind of code can be particularly useful when you want to process or aggregate data from a collection (like a list of books) and collect distinct values (like author names) without duplicates. This operation is commonly used in data processing tasks, such as when preparing data for analysis or reporting. 

### Example Usage

Here's how it might be used in context:

```python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},
    {"title": "Book Four"},  # No author
    {"title": "Book Five", "author": None},  # No author
]

def unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Usage
for author in unique_authors(books):
    print(author)

# Output:
# Author A
# Author B
```

In this example, the output will display each unique author only once.

In [49]:
# try with Streaming
stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    stream=True
)

In [51]:
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    if chunk is None:
        continue
    elif len(chunk.choices) == 0:
        continue
    elif chunk.choices[0].delta.content is not None:
        response += chunk.choices[0].delta.content 
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

Certainly! Let's break down the code snippet:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation

1. **Set Comprehension**: 
   - The code utilizes a **set comprehension** wrapped in curly braces `{}`. This means that it generates a set of unique values derived from the expression inside. 
   - The expression `book.get("author")` retrieves the value associated with the key `"author"` from each dictionary (`book`) in the iterable `books`.

2. **Iteration Over `books`**: 
   - `for book in books` iterates over a collection called `books`, which is expected to be a list (or another iterable) of dictionaries, where each dictionary represents a book.

3. **Conditional Filtering**: 
   - `if book.get("author")` is a condition that filters the results. This means that only books that have an author (i.e., where `book.get("author")` is not `None` or an empty string) are included in the set comprehension.
   
4. **Yielding Values**: 
   - The expression `yield from` is a way to yield all the values from an iterable. It essentially takes each element produced by the expression on the right side and yields it one by one.
   - In this case, the iterable is the set created by the comprehension, which contains the unique authors from the provided `books` collection.

### Result
Overall, this code effectively gathers all unique authors from a list of books (each represented by a dictionary) and yields them one by one. If a book does not have an author (returns `None` or an empty string), it will be excluded from the final set. 

This can be particularly useful in cases where you want to process or display a list of unique authors without duplication.

### Example
For instance, consider the following input:
python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4"},
    {"title": "Book 5", "author": None}
]

The code would yield:
- "Author A"
- "Author B"

As a result, the yielded set would be `{"Author A", "Author B"}`.

In [57]:
# Get Llama 3.2 to answer
import ollama

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

**Code Explanation**
=====================

The given code snippet is a generator expression that uses the `yield from` syntax to delegate iteration to another iterable.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's a breakdown of what's happening:

1. `{...}`: This is an immediately invoked dictionary comprehension, also known as a nested dictionary literal.
2. `for book in books`: This loops through each item in the `books` collection.
3. `if book.get("author")`: This filters out any items that don't have a "author" key or value.
4. `book.get("author")`: For each remaining item, this extracts its associated author value.

**What is `yield from`?**
-------------------------

The `yield from` syntax was introduced in Python 3.3. It allows you to delegate iteration to another iterable, such as a generator or an iterator, rather than writing out the entire iteration manually.

In this case, the outer dictionary comprehension would iterate over each book and try to extract its author value. However, by using `yield from`, we can offload the iteration to the inner expression `{...}`, which is likely another generator expression that yields a sequence of authors.

**Example Use Case**
--------------------

Suppose we have a list of books, where each book has an "author" key:
```python
books = [
    {"title": "Book 1", "author": "John Doe"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Jane Smith"}
]
```
If we want to extract the authors from these books, we can use the given code:
```python
authors = yield from {book.get("author") for book in books if book.get("author")}
print(authors)  # Output: ["John Doe", "Jane Smith"]
```
By using `yield from`, we can make the code more concise and easier to read, especially when dealing with complex iteration scenarios.

In [61]:
# Alternative way to call Llama 3.2 via OpenAI API compatible interface

ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages
)

display(Markdown(response.choices[0].message.content))


**Code Breakdown**
================

The given line of code is a Python expression that utilizes the `yield from` syntax, which was introduced in Python 3.3 as part of the generator expression.

```python
yield from {book.get("author") for book in books if book.get("author")}
```

Here's what it does:

1. **List Comprehension**: The code starts with a list comprehension, `book.get("author") for book in books`. This generates a new list that contains the result of calling `get` on each dictionary (`book`) in the `books` collection.
2. **`yield from`**: The `yield from` syntax is used to delegate the iterations over some sequence (in this case, a generator expression) to an existing generator.

**What it does:**

When you call `$()` on a Python object that implements the iterator protocol, like a list or tuple, Python will yield each element in turn. Adding `yield from ...` essentially "unpacks" its arguments and yields their own iterables.

In this case, `yield from...` is used to create a new generator expression that produces all elements returned by the existing dictionary comprehension. This means the entire flow of iteration is delegated to the `get("author") for book in books if book.get("author")` expression itself.

However, since there's no actual iterable here, and we're only dealing with a single list (`books`), this approach doesn't add significant functionality. One possible interpretation could be to create an iterator over a specific subset of the data - in this case, all dictionaries that contain an `"author"` key.

**Why**
-----------

This code can be useful when:

*   You need to process part of a collection (e.g., only dictionaries with a certain attribute or value).
*   You have a chain of filter operations where you want them performed sequentially.
*   When working with large data collections and optimizing memory usage by avoiding unnecessary list comprehensions.

It ensures that the computation flows in a specific order: filtering, then extracting the desired data. This can be especially beneficial for lazy evaluation cases where generating intermediate steps might have performance implications.